## Puntos calientes
Idealista tiene datos de sala de prensa. Mediante estos datos podemos obtener un historico de los precios por municipio y distritos y la ultima información de los barrios:

* Precio m2 (último mes)
* Variación mensual
* Variación trimestral
* Variación anual
* Máximo histórico
* Variación máxima

### Prueba de concepto
Antes de entrar en detalle al código vamos a realizar una prueba de concepto para indicar como funciona el proceso.

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup # Web Scrapping
import time # Prevenir detección de Scrapping
import traceback # Visualización de excepciones
from selenium import webdriver
import random
import os

In [2]:
LOCATION_COL = "Localización"
PRICE_COL = "Precio m2"
PERCENTAGES_COL =  ["Variación mensual", "Variación trimestral", "Variación anual", "Variación máximo"]
TIME_BETWEEN_RQ = [2, 10] # Rango de tiempo de espera entre peticiones
TIME_BEFORE_RQ = [4, 8] # Tango de tiempo de espera en cargar la página
BASE_URI = "https://www.idealista.com"

**IMPORTANTE:** Hay varias maneras de conseguir el HTML uno de ellos es con selenium, el cual replica la navegación, debido a que queremos evitar varios sockets abiertos y problemas con el scrapeo se usa este. 

In [3]:
driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")

In [5]:
# Web with the municipality information
url = 'https://www.idealista.com/sala-de-prensa/informes-precio-vivienda/venta/madrid-comunidad/madrid-provincia/madrid/'

driver.get(url)
tables = pd.read_html(driver.page_source)

Dentro de un mismo html puede haber múltiples tablas, mediante parámetros en la función podemos filtrar las tablas que lee. Debido a que conocemos la web y sabemos la posición en la que se encuentra la tabla no hará falta.

In [6]:
print(f'Numero tablas: {len(tables)}')

Numero tablas: 2


In [7]:
df = tables[0].copy()
df

,Localización,Precio m2 ago 2021,Variación mensual,Variación trimestral,Variación anual,Máximo histórico,Variación máximo
0,Madrid,3.733 €/m2,"- 0,7 %","- 0,4 %","+ 1,4 %",3.822 €/m2 jul 2019,"- 2,3 %"
1,Arganzuela,3.986 €/m2,"- 0,3 %","- 0,3 %","+ 1,9 %",4.096 €/m2 jul 2019,"- 2,7 %"
2,Barajas,3.340 €/m2,"+ 3,6 %","+ 6,7 %","+ 6,8 %",3.663 €/m2 mar 2009,"- 8,8 %"
3,Carabanchel,2.107 €/m2,"- 0,8 %","- 2,2 %","- 3,0 %",3.173 €/m2 jun 2007,"- 33,6 %"
4,Centro,4.827 €/m2,"+ 0,7 %","+ 0,5 %","- 2,7 %",5.096 €/m2 ene 2020,"- 5,3 %"
5,Chamartín,5.110 €/m2,"- 0,5 %","- 0,1 %","+ 0,8 %",5.216 €/m2 nov 2018,"- 2,0 %"
6,Chamberí,5.347 €/m2,"0,0 %","+ 1,9 %","+ 0,3 %",5.432 €/m2 mayo 2020,"- 1,6 %"
7,Ciudad Lineal,2.981 €/m2,"- 0,8 %","+ 1,2 %","+ 1,2 %",3.578 €/m2 oct 2007,"- 16,7 %"
8,Fuencarral,3.536 €/m2,"+ 0,9 %","+ 0,5 %","+ 0,6 %",3.726 €/m2 mayo 2008,"- 5,1 %"
9,Hortaleza,3.836 €/m2,"+ 0,6 %","+ 2,2 %","+ 5,2 %",3.836 €/m2 ago 2021,"0,0 %"


Ahora que ya tenemos la talba vamos a crearnos unas funciones que limpien el formato de los datos. La columna de Máximo histórico no se mirara por ahora.

In [8]:
def remove_price_m(price: str):
    """
    Transform text price format to int
    
    :param price: Str con el precio en formato texto
    :return: price in int format
    """
    if(price != "n.d."): # If there is price
        return int(''.join(filter(str.isdigit, price[:-4])))

def remove_percentage(percentage: str):
    """
    Transform text percentage format to float
    
    :param percentage
    :return: percentage in dobule format or nan
    """
    if any(char.isdigit() for char in percentage):
        return float(percentage.replace(" ", "").replace(",", ".")[:-1])
    return float('nan')

def rename_price_column(df):
    """
    Remove date from price name column
    
    :param df: DataFrame to process
    :return: DataFrame with the columns renamed
    """
    column_to_rename = [column for column in df.columns if "Precio m2" in column][0]
    return df.rename(columns={column_to_rename: PRICE_COL})

df = rename_price_column(df)

for column in PERCENTAGES_COL:
    df[column] = df[column].apply(lambda x: remove_percentage(x))

df[PRICE_COL] = df[PRICE_COL].apply(lambda x: remove_price_m(x))

In [9]:
df

,Localización,Precio m2,Variación mensual,Variación trimestral,Variación anual,Máximo histórico,Variación máximo
0,Madrid,3733,-0.7,-0.4,1.4,3.822 €/m2 jul 2019,-2.3
1,Arganzuela,3986,-0.3,-0.3,1.9,4.096 €/m2 jul 2019,-2.7
2,Barajas,3340,3.6,6.7,6.8,3.663 €/m2 mar 2009,-8.8
3,Carabanchel,2107,-0.8,-2.2,-3.0,3.173 €/m2 jun 2007,-33.6
4,Centro,4827,0.7,0.5,-2.7,5.096 €/m2 ene 2020,-5.3
5,Chamartín,5110,-0.5,-0.1,0.8,5.216 €/m2 nov 2018,-2.0
6,Chamberí,5347,0.0,1.9,0.3,5.432 €/m2 mayo 2020,-1.6
7,Ciudad Lineal,2981,-0.8,1.2,1.2,3.578 €/m2 oct 2007,-16.7
8,Fuencarral,3536,0.9,0.5,0.6,3.726 €/m2 mayo 2008,-5.1
9,Hortaleza,3836,0.6,2.2,5.2,3.836 €/m2 ago 2021,0.0


### Automatización
Ahora que ya he hemos visto como se procesa la tabla vamos a ver como podríamos atuomatizar el proceso para obtener todos los distritos y barrios de Madrid.

In [10]:
def process_table(df):
    """
    Format df
    
    :param df: DataFrame to format
    """
    df_columns = df.columns
    
    df = rename_price_column(df)
    
    for column in PERCENTAGES_COL:
        if column in df_columns:
            df[column] = df[column].apply(lambda x: remove_percentage(x))
        
    df[PRICE_COL] = df[PRICE_COL].apply(lambda x: remove_price_m(x))
    
    return df


def get_page(url: str):
    """
    Go to URL
    
    :param url: str with the url to go
    """
    # Wait to avoid too much req/s
    time.sleep(random.randrange(TIME_BETWEEN_RQ[0], TIME_BETWEEN_RQ[1]))
    driver.get(url)
    # Wait load page
    time.sleep(random.randrange(TIME_BEFORE_RQ[0], TIME_BEFORE_RQ[1])) 
    
    
def click_a_tag(element: str, value: str):
    """
    Click on a tag element
    
    :param element: element to filter in a tag
    :param value: value to match with the element
    """
    # Wait to avoid too much req/s
    time.sleep(random.randrange(TIME_BETWEEN_RQ[0], TIME_BETWEEN_RQ[1]))
    
    historic_ref = driver.find_element_by_xpath(f"//a[@{element}='{value}']")
    driver.execute_script("arguments[0].scrollIntoView({block: \"center\", inline: \"center\"});", historic_ref)
    time.sleep(3)
    historic_ref.click()
    
    # Wait load page
    time.sleep(random.randrange(TIME_BEFORE_RQ[0], TIME_BEFORE_RQ[1])) 

def go_back(n: int = 1):
    """
    Go back in navigation
    
    :param n: times to go back
    """
    for i in range(0, n):
        time.sleep(random.randrange(TIME_BEFORE_RQ[0], TIME_BEFORE_RQ[1]))
        driver.back()

    
def save_distribution(has_links: bool = True):
    """
    Save child information into csv
    
    :param has_links: if the childs have childs they have a link that is need to be saved
    :return: DataFrame with the information formated
    """
    html_table = BeautifulSoup(driver.page_source).findAll('table')[0]
    df = pd.read_html(str(html_table))[0]
    if has_links:
        if not any(col in PERCENTAGES_COL for col in df.columns): # Checkeamos talba is correct
            html_table = BeautifulSoup(driver.page_source).findAll('table')[1]
            df = pd.read_html(str(html_table))[0]

        # Add None at first element beacuse has not link
        df['Link'] = ["None"] + [BASE_URI + link.get('href') for link in html_table.find_all('a')]
           

    processed_df = process_table(df)
    file_name = driver.current_url.split('/')[-2] + "_distribución.csv"
    processed_df.to_csv(file_name, sep='\t', encoding='utf-8')
    
    return processed_df
    

def save_historic():
    """
    Save historic information into csv
    """
    df = pd.read_html(driver.page_source)[0]
    processed_df = process_table(df)
    file_name = driver.current_url.split('/')[-3] + "_historico.csv"
    processed_df.to_csv(file_name, sep='\t', encoding='utf-8')
    
    go_back()

In [11]:
df = pd.read_html(driver.page_source)[0]
df_columns = df.columns
    
df = rename_price_column(df)

for column in PERCENTAGES_COL:
    if column in df_columns:
        df[column] = df[column].apply(lambda x: remove_percentage(x))

df[PRICE_COL] = df[PRICE_COL].apply(lambda x: remove_price_m(x))
df

,Localización,Precio m2,Variación mensual,Variación trimestral,Variación anual,Máximo histórico,Variación máximo
0,Madrid,3733,-0.7,-0.4,1.4,3.822 €/m2 jul 2019,-2.3
1,Arganzuela,3986,-0.3,-0.3,1.9,4.096 €/m2 jul 2019,-2.7
2,Barajas,3340,3.6,6.7,6.8,3.663 €/m2 mar 2009,-8.8
3,Carabanchel,2107,-0.8,-2.2,-3.0,3.173 €/m2 jun 2007,-33.6
4,Centro,4827,0.7,0.5,-2.7,5.096 €/m2 ene 2020,-5.3
5,Chamartín,5110,-0.5,-0.1,0.8,5.216 €/m2 nov 2018,-2.0
6,Chamberí,5347,0.0,1.9,0.3,5.432 €/m2 mayo 2020,-1.6
7,Ciudad Lineal,2981,-0.8,1.2,1.2,3.578 €/m2 oct 2007,-16.7
8,Fuencarral,3536,0.9,0.5,0.6,3.726 €/m2 mayo 2008,-5.1
9,Hortaleza,3836,0.6,2.2,5.2,3.836 €/m2 ago 2021,0.0


In [12]:
municipality_df = save_distribution()
click_a_tag("title", "Ver datos más antiguos")
save_historic()
municipality_df

,Localización,Precio m2,Variación mensual,Variación trimestral,Variación anual,Máximo histórico,Variación máximo,Link
0,Madrid,3733,-0.7,-0.4,1.4,3.822 €/m2 jul 2019,-2.3,None
1,Arganzuela,3986,-0.3,-0.3,1.9,4.096 €/m2 jul 2019,-2.7,https://www.idealista.com/sala-de-prensa/infor...
2,Barajas,3340,3.6,6.7,6.8,3.663 €/m2 mar 2009,-8.8,https://www.idealista.com/sala-de-prensa/infor...
3,Carabanchel,2107,-0.8,-2.2,-3.0,3.173 €/m2 jun 2007,-33.6,https://www.idealista.com/sala-de-prensa/infor...
4,Centro,4827,0.7,0.5,-2.7,5.096 €/m2 ene 2020,-5.3,https://www.idealista.com/sala-de-prensa/infor...
5,Chamartín,5110,-0.5,-0.1,0.8,5.216 €/m2 nov 2018,-2.0,https://www.idealista.com/sala-de-prensa/infor...
6,Chamberí,5347,0.0,1.9,0.3,5.432 €/m2 mayo 2020,-1.6,https://www.idealista.com/sala-de-prensa/infor...
7,Ciudad Lineal,2981,-0.8,1.2,1.2,3.578 €/m2 oct 2007,-16.7,https://www.idealista.com/sala-de-prensa/infor...
8,Fuencarral,3536,0.9,0.5,0.6,3.726 €/m2 mayo 2008,-5.1,https://www.idealista.com/sala-de-prensa/infor...
9,Hortaleza,3836,0.6,2.2,5.2,3.836 €/m2 ago 2021,0.0,https://www.idealista.com/sala-de-prensa/infor...


Ahora que ya disponemos del listado de todos los municipios podemos ir iterando sobre estos y sus repesctivos hijos(barrios). Debido a que se realizará el código del anterior chunk repetidas veces se creara una función.

In [13]:
def get_and_process_page(a_tag_title: str, has_links: bool = True):
    """
    Go to page with a click and process the information
    
    :param a_tag_title: a tag title to click
    :param has_links: the table to process has links 
    """
    click_a_tag("title", a_tag_title)
    df = save_distribution(has_links)
    click_a_tag("title", "Ver datos más antiguos")
    save_historic()   
    return df

def is_processed(a_tag_title: str):
    """
    Checks if a page has been processed before
    
    :param a_tag_title: a tag title to search the element which has to be chekced
    :return: true if has the distribution and historic file, else false
    """
    name = driver.find_element_by_xpath(f"//a[@title='{a_tag_title}']").get_attribute('href').split('/')[-2]
    file_dristribution = name + "_distribución.csv"
    file_historic = name + "_historico.csv"
    return os.path.exists(file_dristribution) and os.path.exists(file_historic)

In [ ]:
for _, district in municipality_df[1:].iterrows(): # First row contains information about parent  
    if not is_processed(district[LOCATION_COL]):
        district_df = get_and_process_page(district[LOCATION_COL], False)
        # Go back to muncipality page
        go_back()